# Welcome to the Development Version of BASS

This notebook is inteded for very advanced users, as there is almost no interactivity features. However, this notebook is all about speed. If you know exactly what you are doing, then this is the notebook for you.

    BASS: Biomedical Analysis Software Suite for event detection and signal processing.
    Copyright (C) 2015  Abigail Dobyns

    This program is free software: you can redistribute it and/or modify
    it under the terms of the GNU General Public License as published by
    the Free Software Foundation, either version 3 of the License, or
    (at your option) any later version.

    This program is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    GNU General Public License for more details.

    You should have received a copy of the GNU General Public License
    along with this program.  If not, see <http://www.gnu.org/licenses/>

In [ ]:
from bass import *

**WARNING** All strings should be raw, especially if in Windows.
    
    r'String!'

In [ ]:
#initialize new file
Data = {}
Settings = {}
Results ={}

############################################################################################
#manual Setting block
Settings['folder']= r"/Users/abigaildobyns/Desktop"
Settings['Label'] = r'rat34_ECG.txt'
Settings['Output Folder'] = r"/Users/abigaildobyns/Desktop/demo"

#transformation Settings
Settings['Absolute Value'] = True #Must be True if Savitzky-Golay is being used

Settings['Bandpass Highcut'] = r'none' #in Hz
Settings['Bandpass Lowcut'] = r'none' #in Hz
Settings['Bandpass Polynomial'] = r'none' #integer

Settings['Linear Fit'] = False #between 0 and 1 on the whole time series
Settings['Linear Fit-Rolling R'] = 0.75 #between 0 and 1
Settings['Linear Fit-Rolling Window'] = 1000 #window for rolling mean for fit, unit is index not time
Settings['Relative Baseline'] = 0 #default 0, unless data is normalized, then 1.0. Can be any float

Settings['Savitzky-Golay Polynomial'] = 4 #integer
Settings['Savitzky-Golay Window Size'] = 251 #must be odd. units are index not time

#Baseline Settings
Settings['Baseline Type'] = r'static' #'linear', 'rolling', or 'static'
#For Linear
Settings['Baseline Start'] = 0.0 #start time in seconds
Settings['Baseline Stop'] = 1.0 #end time in seconds
#For Rolling
Settings['Rolling Baseline Window'] = r'none' #leave as 'none' if linear or static

#Peaks
Settings['Delta'] = 0.25
Settings['Peak Minimum'] = -1 #amplitude value
Settings['Peak Maximum'] = 1 #amplitude value

#Bursts
Settings['Burst Area'] = False #calculate burst area
Settings['Exclude Edges'] = True #False to keep edges, True to discard them
Settings['Inter-event interval minimum (seconds)'] = 0.0100 #only for bursts, not for peaks
Settings['Maximum Burst Duration (s)'] = 10 
Settings['Minimum Burst Duration (s)'] = 0
Settings['Minimum Peak Number'] = 1 #minimum number of peaks/burst, integer
Settings['Threshold']= 0.15 #linear: proportion of baseline. 
                           #static: literal value.
                           #rolling, linear ammount grater than rolling baseline at each time point.

#Outputs
Settings['Generate Graphs'] = False #create and save the fancy graph outputs


#Settings that you should not change unless you are a super advanced user:
#These are settings that are still in development
Settings['Graph LCpro events'] = False
Settings['File Type'] = r'Plain' #'LCPro', 'ImageJ', 'SIMA', 'Plain', 'Morgan'
Settings['Milliseconds'] = False
############################################################################################

In [ ]:
#Load in a Settings File
#initialize new file
Data = {}
Settings = {}
Results ={}

############################################################################################
#manual Setting block
Settings['folder']= "/Users/abigaildobyns/Desktop/Neuron Modeling/morgan voltage data"
Settings['Label'] = 'voltage-TBModel-sec1320-eL-IP0_9.txt'
Settings['Output Folder'] = "/Users/abigaildobyns/Desktop/Neuron Modeling/morgan voltage data/IP0_9"
Settings['File Type'] = 'Morgan' #'LCPro', 'ImageJ', 'SIMA', 'Plain', 'Morgan'
Settings['Milliseconds'] = True

#Load a Settings file
Settings['Settings File'] = '/Users/abigaildobyns/Desktop/Neuron Modeling/morgan voltage data/IP0_9/voltage-TBModel-sec1320-eL-IP0_9.txt_Settings.csv'
Settings = load_settings(Settings)

In [ ]:
Data, Settings, Results = analyze(Data, Settings, Results)

In [ ]:
#plot raw data
plot_rawdata(Data)

In [ ]:
#grouped summary for peaks
Results['Peaks-Master'].groupby(level=0).describe()

In [ ]:
#grouped summary for bursts
Results['Bursts-Master'].groupby(level=0).describe()

In [ ]:
#Call one time series by Key
key = 'Mean1'
graph_ts(Data, Settings, Results, key)

In [ ]:
#raw and transformed event plot
key = 'Mean1'
start =100 #start time in seconds
end= 101#end time in seconds
results_timeseries_plot(key, start, end, Data, Settings, Results)

In [ ]:
#Frequency plot
event_type = 'Peaks'
meas = 'Intervals'
key = 'Mean1' #'Mean1' default for single wave
frequency_plot(event_type, meas, key, Data, Settings, Results)

In [ ]:
#Get average plots, display only
event_type = 'peaks'
meas = 'Peaks Amplitude'
average_measurement_plot(event_type, meas,Results)

In [ ]:
#raster
raster(Results)

In [ ]:
#Batch
event_type = 'Peaks'
meas = 'all'
Results = poincare_batch(event_type, meas, Data, Settings, Results)
pd.concat({'SD1':Results['Poincare SD1'],'SD2':Results['Poincare SD2']})

In [ ]:
#quick poincare
event_type = 'Peaks'
meas = 'Intervals'
key = 'Mean1'
poincare_plot(Results[event_type][key][meas])

In [ ]:
#PSD of DES
Settings['PSD-Event'] = Series(index = ['Hz','ULF', 'VLF', 'LF','HF','dx'])
#Set PSD ranges for power in band

Settings['PSD-Event']['hz'] = 4.0 #freqency that the interpolation and PSD are performed with.
Settings['PSD-Event']['ULF'] = 0.03 #max of the range of the ultra low freq band. range is 0:ulf
Settings['PSD-Event']['VLF'] = 0.05 #max of the range of the very low freq band. range is ulf:vlf
Settings['PSD-Event']['LF'] = 0.15 #max of the range of the low freq band. range is vlf:lf
Settings['PSD-Event']['HF'] = 0.4 #max of the range of the high freq band. range is lf:hf. hf can be no more than (hz/2)
Settings['PSD-Event']['dx'] = 10 #segmentation for the area under the curve. 

event_type = 'Peaks'
meas = 'Intervals'
key = 'Mean1'
scale = 'raw'
Results = psd_event(event_type, meas, key, scale, Data, Settings, Results)
Results['PSD-Event'][key]

In [ ]:
#PSD of raw signal
#optional
Settings['PSD-Signal'] = Series(index = ['ULF', 'VLF', 'LF','HF','dx'])
#Set PSD ranges for power in band

Settings['PSD-Signal']['ULF'] = 100 #max of the range of the ultra low freq band. range is 0:ulf
Settings['PSD-Signal']['VLF'] = 200 #max of the range of the very low freq band. range is ulf:vlf
Settings['PSD-Signal']['LF'] = 300 #max of the range of the low freq band. range is vlf:lf
Settings['PSD-Signal']['HF'] = 400 #max of the range of the high freq band. range is lf:hf. hf can be no more than (hz/2)
Settings['PSD-Signal']['dx'] = 10 #segmentation for the area under the curve. 

scale = 'raw' #raw or db
Results = psd_signal(version = 'original', key = 'Mean1', scale = scale, 
                     Data = Data, Settings = Settings, Results = Results)
Results['PSD-Signal']

In [ ]:
#spectrogram
version = 'original'
key = 'Mean1'
spectogram(version, key, Data, Settings, Results)

In [ ]:
#Moving Stats
event_type = 'Peaks'
meas = 'all'
window = 60 #seconds
Results = moving_statistics(event_type, meas, window, Data, Settings, Results)

In [ ]:
#Histogram Entropy-events
event_type = 'Bursts'
meas = 'all'
Results = histent_wrapper(event_type, meas, Data, Settings, Results)
Results['Histogram Entropy']

## pyEEG required for approximate and sample entropy

In [ ]:
#Approximate Entropy-events
event_type = 'Peaks'
meas = 'all'
Results = ap_entropy_wrapper(event_type, meas, Data, Settings, Results)
Results['Approximate Entropy']

In [ ]:
#Sample Entropy-events
event_type = 'Peaks'
meas = 'all'
Results = samp_entropy_wrapper(event_type, meas, Data, Settings, Results)
Results['Sample Entropy']

In [ ]:
#Approximate Entropy on raw signal
#takes a VERY long time
from pyeeg import ap_entropy

version = 'original' #original, trans, shift, or rolling
key = 'Mean1' #Mean1 default key for one time series
start = 0 #seconds, where you want the slice to begin
end = 1 #seconds, where you want the slice to end. The absolute end is -1

ap_entropy(Data[version][key][start:end].tolist(), 2, (0.2*np.std(Data[version][key][start:end])))

In [ ]:
#Sample Entropy on raw signal
#takes a VERY long time
from pyeeg import samp_entropy

version = 'original' #original, trans, shift, or rolling
key = 'Mean1' #Mean1 default key for one time series
start = 0 #seconds, where you want the slice to begin
end = 1 #seconds, where you want the slice to end. The absolute end is -1

samp_entropy(Data[version][key][start:end].tolist(), 2, (0.2*np.std(Data[version][key][start:end])))

## Need help?

Try checking the docstring of a function you are struggling with.
    
    moving_statistics?
    help(moving_statistics)

In [ ]:
moving_statistics?

In [ ]:
import pyeeg

In [ ]:
pyeeg.samp_entropy?